## Agent - 2

在上一节的基础示例中，完整的展示了Agent所需要的元素

在Agent中，需要了解几个关键概念

### Schema

LangChain 有几个抽象的概念，可以使得使用代理变得容易

#### AgentAction

**这是一个数据类，表示代理应该采取的操作。它有一个 tool 属性(应该调用的工具的名称)和一个 tool _ input 属性(该工具的输入)**

#### AgentFinish

这表示代理准备返回给用户时的最终结果。它包含 return_values 键值映射，该映射包含最终的代理输出。通常，这包含一个输出键，其中包含作为代理响应的字符串。

#### Intermediate Steps
它们表示以前的代理操作以及此 CURRENT 代理运行的相应输出。

这些对于传递到未来的迭代很重要，这样代理就知道它已经完成了什么工作。类型为 List [ Tuple [ AgentAction，Any]。需要注意，当前的观察类型仍然是任意类型，以便最大限度地保持灵活性。实际上，这通常是一个字符串。

### Agent

这是负责决定下一步采取什么动作的类。 这是由语言模型和提示驱动的。 该提示可以包括以下内容：

- 代理的个性（对于以某种方式响应很有用）

- 代理的背景上下文（对于给予其更多关于所要求完成的任务类型的上下文很有用）

- 调用更好推理的提示策略（最著名/广泛使用的是ReAct）
LangChain提供了几种不同类型的代理来入门。

#### Agent Inputs

代理的输入是键-值映射。只有一个必需的键: intermediate_steps，它对应于上面描述的 Intermediate Step。

通常，PromptTemplate 负责将这些对转换为能够最好地传递到 LLM 中的格式。

#### Agent Outputs

输出是要执行的下一个操作或要发送给用户的最终响应(AgentActions 或 AgentFinish)。具体来说，这可以类型为 Union [ AgentAction，List [ AgentAction ] ，AgentFinish ]。

输出解析器负责获取原始 LLM 输出并将其转换为以下三种类型之一。

### AgentExecutor
代理执行器

代理执行器是代理的运行时。 这是实际调用代理并执行其选择的动作的部分。 以下是此运行时的伪代码：

```javascript
next_action = agent.get_action(...)
while next_action != AgentFinish:
    observation = run(next_action)
    next_action = agent.get_action(..., next_action, observation)
return next_action
```

虽然这看起来很简单，但此运行时为您处理了几个复杂性，包括：

- 处理代理选择不存在的工具的情况
- 处理工具发生错误的情况
- 处理代理生成无法解析为工具调用的输出的情况
- 在所有级别上记录和可观察性（代理决策，工具调用）-可以输出到stdout或LangSmith

### Tools

LangChain 提供了大量的内置工具，但是也使得定义自己的工具变得容易(包括自定义描述)。有关内置工具的完整列表，请参见工具集成部分[https://python.langchain.com/v0.1/docs/modules/tools/]

工具是代理调用的函数，提供一下功能：

- 给代理访问正确工具的权限
- 以对代理最有帮助的方式**描述**工具

如果没有这两者，您想要构建的代理将无法工作。 如果您不给代理访问正确工具的权限，它将永远无法完成目标。 如果您不正确描述工具，代理将不知道如何正确使用它们。

### Toolkits

代理可以访问的工具集合通常比单个工具更重要。 

为此，LangChain提供了工具包的概念-用于实现特定目标所需的一组工具。 通常一个工具包中有3-5个工具。

LangChain提供了一系列广泛的工具包来入门。 有关工具包的完整列表，请参见这里[https://python.langchain.com/v0.1/docs/integrations/toolkits/]